In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('train.csv')

In [10]:
df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [11]:
df.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
743,0,3,male,24.0,1,0,16.1000,S
251,0,3,female,29.0,1,1,10.4625,S
20,0,2,male,35.0,0,0,26.0000,S
324,0,3,male,NaN,8,2,69.5500,S
123,1,2,female,32.5,0,0,13.0000,S
661,0,3,male,40.0,0,0,7.2250,C
128,1,3,female,NaN,1,1,22.3583,C
756,0,3,male,28.0,0,0,7.7958,S
718,0,3,male,NaN,0,0,15.5000,Q
580,1,2,female,25.0,1,1,30.0000,S


In [12]:
df.isnull().sum()

,0
Survived,0
Pclass,0
Sex,0
Age,177
SibSp,0
Parch,0
Fare,0
Embarked,2


In [ ]:
from sklearn.impute import SimpleImputer



In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop('Survived',axis=1),df['Survived'],test_size=0.2,random_state=42)

In [16]:
# applying Imputation

from sklearn.impute import SimpleImputer

Si_age = SimpleImputer(strategy='mean')
Si_embarked = SimpleImputer(strategy='most_frequent')

X_train_age= Si_age.fit_transform(X_train[['Age']])
X_train_embarked = Si_embarked.fit_transform(X_train[['Embarked']])

X_test_age = Si_age.transform(X_test[['Age']])
X_test_embarked = Si_embarked.transform(X_test[['Embarked']])

In [19]:
# one hot encoding Sex and embarked

from sklearn.preprocessing import OneHotEncoder

ohe_sex = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)


In [22]:
X_train['Sex']

,Sex
331,male
733,male
382,male
704,male
813,female
...,...
106,female
270,male
860,male
435,female


In [20]:
X_train_sex

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [23]:
X_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [24]:
X_train_remain = X_train.drop(columns=['Sex','Age','Embarked'])

In [27]:
X_test_remain = X_test.drop(columns=['Sex','Age','Embarked'])

In [26]:
X_train_remain

,Pclass,SibSp,Parch,Fare
331,1,0,0,28.5000
733,2,0,0,13.0000
382,3,0,0,7.9250
704,3,1,0,7.8542
813,3,4,2,31.2750
...,...,...,...,...
106,3,0,0,7.6500
270,1,0,0,31.0000
860,3,2,0,14.1083
435,1,1,2,120.0000


In [31]:
X_train_transformed = np.concatenate((X_train_remain,X_train_age,X_train_sex,X_train_embarked),axis=1)

In [30]:
X_test_transformed = np.concatenate((X_test_remain,X_test_age,X_test_sex,X_test_embarked),axis=1)

In [33]:
X_train_transformed.shape

(712, 10)

In [34]:
X_train.shape

(712, 7)

In [35]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train_transformed,y_train)


DecisionTreeClassifier()

In [37]:
y_pred = dtc.predict(X_test_transformed)
y_pred

array([0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1])

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.7821229050279329

In [42]:
import pickle
pickle.dump(ohe_sex,open('ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('ohe_embarked.pkl','wb'))
pickle.dump(dtc,open('dtc.pkl','wb'))

In [40]:
ohe_embarked

OneHotEncoder(handle_unknown='ignore', sparse_output=False)